In [1]:
import pandas as pd


In [2]:
# Load the CSV files into Pandas DataFrames
entry_data = pd.read_csv('SA_entry_data.csv', parse_dates=['entry_date'])
signup_data = pd.read_csv('SA_signup_data.csv', parse_dates=['reg_date'])

In [3]:
# Merge the two DataFrames on user_id and filter rows based on the reg_date
merged_data = pd.merge(entry_data, signup_data, how='inner', on='user_id')
filtered_data = merged_data[(merged_data['reg_date'] >= '2020-09-01') & (merged_data['reg_date'] <= '2020-09-30')]


In [4]:
# Calculate the total entry amount for members who signed up in September 2020
total_entry_amount = filtered_data['entry_amount'].sum()
total_entry_amount

1495076.5

In [5]:
# Filter the entry_data to include only entries from September 2020
september_entries = entry_data[(entry_data["entry_date"] >= "2020-09-01") & (entry_data["entry_date"] <= "2020-09-30")]

# Add a new column to the DataFrame representing the day of the week (Monday=0, Sunday=6)
september_entries["day_of_week"] = september_entries["entry_date"].dt.dayofweek

# Group the data by day_of_week and user_id, and calculate the average entry amount per member for each day of the week
average_entry_per_member = september_entries.groupby("day_of_week")["entry_amount"].mean().reset_index()

# Map the numerical day_of_week values to their corresponding names
day_name_mapping = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}
average_entry_per_member["day_of_week"] = average_entry_per_member["day_of_week"].map(day_name_mapping)

# Rename columns for better presentation
average_entry_per_member.columns = ["Day of Week", "Average Entry Amount per Member"]

average_entry_per_member

,Day of Week,Average Entry Amount per Member
0,Monday,310.576259
1,Tuesday,315.213912
2,Wednesday,313.205777
3,Thursday,312.127630
4,Friday,312.961514
5,Saturday,315.788505
6,Sunday,308.977429


In [6]:
# Filter the entry_data to include only entries from September 2020
september_entries = entry_data[(entry_data["entry_date"] >= "2020-09-01") & (entry_data["entry_date"] <= "2020-09-30")]

# Group the data by player and league, and calculate the total entry amount for each player
top_players = september_entries.groupby(["player", "league"])["entry_amount"].sum().reset_index()

# Sort the result by total entry amount in descending order and select the top 5 players
top_players = top_players.sort_values(by="entry_amount", ascending=False).head(5)

# Rename columns for better presentation
top_players.columns = ["Player", "League", "Total Entry Amount"]

top_players

,Player,League,Total Entry Amount
85,Jamal Murray,NBA,165348.0
149,Nikola Jokic,NBA,163722.0
121,LeBron James,NBA,162286.5
94,Jayson Tatum,NBA,159838.5
98,Jimmy Butler,NBA,151018.0


In [8]:
# Calculate the total entry amount for each player
total_entry_amount_per_player = entry_data.groupby("player")["entry_amount"].sum().reset_index()
total_entry_amount_per_player.columns = ["player", "total_entry_amount"]

# Calculate each user's entry amount for each player
user_contribution_per_player = entry_data.groupby(["player", "user_id"])["entry_amount"].sum().reset_index()
user_contribution_per_player.columns = ["player", "user_id", "user_entry_amount"]

# Merge the two DataFrames to calculate each user's contribution (by percentage) to the total entry amount for each player
user_contribution_percentage = pd.merge(user_contribution_per_player, total_entry_amount_per_player, on="player")
user_contribution_percentage["share"] = user_contribution_percentage["user_entry_amount"] / user_contribution_percentage["total_entry_amount"] * 100

# Select the desired columns for the final output: player_name, user_id, and share
final_output = user_contribution_percentage[["player", "user_id", "share"]]

# Rename columns for better presentation
final_output.columns = ["Player Name", "User ID", "Share"]

final_output.head()  # Displaying only the first five rows of the result


,Player Name,User ID,Share
0,A.J. Brown,03b87df7-fffb-49c7-aba6-617a32ba3c42,3.223042
1,A.J. Brown,0fbc0c67-4cdb-495a-85d2-bb1e88c94ffa,5.850988
2,A.J. Brown,1635acc1-0a91-4fbf-9955-ee612d914585,4.613187
3,A.J. Brown,1c3afe82-7c86-45de-abfc-c0b99cf25e23,3.346822
4,A.J. Brown,23f1b787-91f0-4bac-9d42-20103abbd05e,0.780766


In [9]:
final_output.tail()  # Displaying only the last five rows of the result

,Player Name,User ID,Share
12473,Zach Ertz,f62e9591-8bc9-4cb7-9889-61e42a9e5eeb,1.411730
12474,Zach Ertz,fa6dca68-6e21-4768-bf47-bcdb95a9a93e,1.404454
12475,Zach Ertz,fc343f1d-1812-477c-88d8-8a3b21731d5a,2.947169
12476,Zach Ertz,fd718ab5-5b37-4ba9-a6ed-96c71124a43f,1.423859
12477,Zach Ertz,fe3acd32-f6be-462f-a291-de9ddbd77a39,0.383253
